In [5]:
import pandas as pd
import librosa
import os
import validators

In [27]:
metadata = pd.read_csv('OADTM2205-raw.csv')
#remove skill_name is NaN and skill_name is olli.others
metadata = metadata[~metadata['skill_name'].isna()]
metadata = metadata[metadata['skill_name'] != 'olli.others']
metadata.to_csv('metadata/raw_multiskill.csv', index=False)

In [15]:
metadata2 = pd.read_csv('OADTM2205-raw.csv')
#remove all skill_name, just keep both NaN and olli.others
metadata3 = metadata2[metadata2['skill_name'].isna()].append(metadata2[metadata2['skill_name'] == 'olli.others'])
metadata3.to_csv('metadata/raw_others.csv', index=False)


C:\Users\admin\AppData\Local\Temp\ipykernel_10768\2751833123.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metadata3 = metadata2[metadata2['skill_name'].isna()].append(metadata2[metadata2['skill_name'] == 'olli.others'])


In [3]:
csv2 = pd.read_csv('metadata/raw_multiskill.csv')

In [4]:
csv2['skill_name'].value_counts()

public             362599
olli.playmusic     118709
iot                 24936
olli.weather         9992
olli.reminder        4358
Translator           1166
olli.calculator      1045
Riddles               852
Horoscope             271
Đặt vé máy bay          5
Name: skill_name, dtype: int64

In [5]:
#take 10% of data group by skill_name, user_id
csv2_10per = csv2.groupby(['skill_name', 'user_id']).apply(lambda x: x.sample(frac=0.1))
csv2_10per.to_csv('metadata/raw_multiskill_10per.csv', index=False)

In [14]:
csv2_10per = pd.read_csv('metadata/raw_multiskill.csv')
#check user_id is NaN
csv2_10per[csv2_10per['user_id'].isna()]
#check utterance_url by validators.url, print False
csv2_10per[csv2_10per['utterance_url'].apply(lambda x: validators.url(x)) == False]

,utterance,utterance_url,intent_name,skill_name,device_id,user_id
